In [16]:
"""
10-meter aspen patch metrics
author: maxwell.cook@colorado.edu
"""

import os, sys, time
import geopandas as gpd
import pylandstats as pls
import multiprocessing as mp
import concurrent.futures
import numpy as np
import rasterio as rio
from rasterio.mask import mask
from multiprocessing import Pool, cpu_count
from tqdm.notebook import tqdm
from shapely.geometry import box

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/')

# Custom functions
sys.path.append(os.path.join(maindir,'aspen-fire/Aim2/code/Python'))
from __functions import *

proj = 'EPSG:5070' # albers

print("Ready to go !")

Ready to go !


In [4]:
fp = os.path.join(projdir, 'Aim3/data/spatial/mod/srm_firesheds_model_data.gpkg')
firesheds = gpd.read_file(fp)
firesheds.columns

Index(['fs_id', 'sfs_id', 'sfs_area_ha', 'fs_name', 'sfs_exposure',
       'pct_disturbed', 'US_L4CODE', 'US_L4NAME', 'trend_area', 'trend_count',
       'historic', 'ssp245', 'ssp585', 'delta245', 'delta585', 'aspen10_pct',
       'aspen10_pixn', 'combust_sum', 'msbf_count_sum', 'pop_density_max',
       'pop_count_sum', 'wui1', 'wui2', 'wui3', 'wui4', 'wui_dist_mean',
       'sfs_area', 'burned_area_c', 'burned_pct_c', 'whp_p95', 'dom_evt1',
       'dom_evt2', 'dom_evt3', 'dom_evt4', 'Aspen', 'Douglas_fir',
       'Gambel_oak', 'Lodgepole', 'Piñon_juniper', 'Ponderosa', 'Sagebrush',
       'Spruce_fir', 'White_fir', 'forest_cc_mean', 'forest_ch_mean',
       'geometry'],
      dtype='object')

In [20]:
firesheds['sfs_area_ha'].describe()

count     1637.000000
mean     10315.122883
std       1246.483878
min       5832.000000
25%       9564.480000
50%      10380.960000
75%      11080.800000
max      14580.000000
Name: sfs_area_ha, dtype: float64

In [8]:
# subset to fireshed with at least ~5% aspen cover
print(len(firesheds))
fs_aspen = firesheds[firesheds['aspen10_pct'] > 0.05]
len(fs_aspen)

1637


1416

In [17]:
# patch analysis - largest patch size, mean patch size, etc

dir = os.path.join(projdir,'Aim1/data/spatial/mod/results/classification/')
aspen10_fp = os.path.join(dir,'s2aspen_distribution_10m_y2019_CookEtAl.tif')

# Define metrics to calculate
cls_metrics = ['number_of_patches', 'patch_density', 'largest_patch_index']

# Function to process a single fireshed grid
def process_fireshed(fireshed):
    """ Process a single fireshed grid for patch statistics. """
    try:
        unit_id = fireshed["sfs_id"]
        geometry = fireshed["geometry"]  # Shapely Polygon

        with rio.open(aspen10_fp) as src:
            # Extract raster subset using rasterio mask (faster than .rio.clip)
            out_image, _ = mask(src, [geometry], crop=True)
            arr = out_image.squeeze()

        # Ensure valid data exists
        if np.all(arr == src.nodata) or np.count_nonzero(arr) == 0:
            return None

        # Compute patch statistics
        ls = pls.Landscape(arr, res=(10, 10))
        patches = ls.compute_class_metrics_df(metrics=cls_metrics)
        patches["sfs_id"] = unit_id

        return patches

    except Exception as e:
        print(f"Error processing grid {fireshed['sfs_id']}: {e}")
        return None

if __name__ == "__main__":
    t0 = time.time()

    num_workers = 2  # adjust as needed
    fs_list = fs_aspen.to_dict(orient="records")  # Convert GeoDataFrame to list

    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = {executor.submit(process_fireshed, fs): fs for fs in fs_list}
        
        for future in tqdm(concurrent.futures.as_completed(futures), 
                           total=len(futures), desc="Processing firesheds"):
            result = future.result()
            if result is not None:
                results.append(result)

    # Merge results into a DataFrame
    patch_metrics_df = pd.concat(results, ignore_index=True) if results else pd.DataFrame()

    t1 = (time.time() - t0) / 60
    print(f"\nTotal elapsed time: {t1:.2f} minutes.\n")

Processing firesheds:   0%|          | 0/1416 [00:00<?, ?it/s]


Total elapsed time: 1.26 minutes.



In [18]:
patch_metrics_df.head()

,number_of_patches,patch_density,largest_patch_index,sfs_id
0,86,3.815372,0.095384,20467
1,2,0.088730,68.277848,20467
2,1941,54.625124,2.078062,20469
3,2,0.056286,59.755270,20469
4,1147,34.018549,2.622127,21247


In [19]:
# save this file out.
out_fp = os.path.join(projdir,'Aim3/data/tabular/firesheds_aspen10_patches.csv')
patch_metrics_df.to_csv(out_fp)
print(f"Saved to: {out_fp}")

Saved to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim3/data/tabular/firesheds_aspen10_patches.csv
